In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from models.tokenization import *
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from models.model_architecture import *
from models.train_eval import *
from tqdm import tqdm, tqdm_notebook
parent_path='../Data/New_Data_15-06-2020/'

In [ ]:
annotated_df=pd.read_pickle(parent_path+'Fearspeech_data_final.pkl')


In [ ]:
torch.cuda.set_device(1)

In [ ]:
params={'model_path':'bert-base-multilingual-cased',
        'preprocess_doc':False,
        'max_length':256,
        'batch_size':32,
        'hidden_size':128,
        'weights':[1.0,1.0],
        'seq_model':'lstm',
        'data_path':parent_path+'Fearspeech_data_final.pkl',
        'max_sentences_per_doc':5,
        'transformer_type':'lstm_transformer',
        'device':'cpu',
        'learning_rate':5e-5,
        'epsilon':1e-8,
        'random_seed':2020,
        'epochs':50,
        'max_memory':0.5
       }


In [ ]:
train_phase(params)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
    

In [ ]:
temp=tokenizer.tokenize("I am goof")



In [ ]:
temp=['<s>']+temp+['</s>']

In [ ]:
tokenizer.convert_tokens_to_ids(temp)

In [ ]:
from transformers import AutoConfig
config=AutoConfig.from_pretrained("xlm-roberta-base")

In [ ]:
config

In [ ]:
annotated_df=pd.read_pickle(params['data_path'])
params_preprocess={'remove_numbers': True, 'remove_emoji': True, 'remove_stop_words': False, 'tokenize': False}
list_sents=[preprocess_sent(ele,params=params_preprocess).strip() for ele in tqdm_notebook(annotated_df['message_text'],total=len(annotated_df))]

In [ ]:
import random 
random.sample(list_sents,5)

In [ ]:
count=[]
for sent in list_sents:
    count.append(len(tokenizer.tokenize(sent)))

X_0 = np.array(list_sents,dtype='object')
y_0 = np.array(annotated_df['one_fear_speech'])
tokenizer = AutoTokenizer.from_pretrained(params['model_path'])
    
X_train = encode_documents(X_0,params,tokenizer) 

In [ ]:
count_greater=0
for ele in count:
    if(ele > (128*5)):
        count_greater+=1
count_greater

In [ ]:
count

In [ ]:
train_dataloader=return_dataloader(X_train,y_0,params,is_train=True)


In [ ]:
train_dataloader

In [ ]:
import torch
import torch.autograd.profiler as profiler
with profiler.profile(record_shapes=True) as prof:
    with profiler.record_function("model_inference"):
        for step, batch in tqdm(enumerate(train_dataloader)):
                b_input_ids = batch[0]
                b_att_masks = batch[1]
                b_token_types = batch[2]
                b_labels = batch[3]
                outputs = model(input_ids=b_input_ids, attention_mask=b_att_masks, token_type_ids=b_token_types,labels=b_labels)
                break




In [ ]:
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

In [ ]:
b_input_ids.shape

In [ ]:
b_input_ids[0][:20,1]

In [ ]:
model=select_transformer_model(params['transformer_type'],params['model_path'],params)

In [ ]:
import torch


In [ ]:
inputs = torch.randn((5, 3, 224, 224),dtype='long')

In [ ]:
inputs